# RCS SQL Intro

# SQL - Structured Query Language
Domain-specific language used in programming and designed for managing data held in a relational database management system (RDBMS)

# History
* Set Theory
* Edgar F. Cobb Relation Model 1970 (later https://en.wikipedia.org/wiki/Codd%27s_12_rules)
* Normalforms (Informally, a relational database relation is often described as "normalized" if it meets third normal form. Most 3NF relations are free of insertion, update, and deletion anomalies.)
* ACID (Atomicity, Consistency, Isolation, Durability) 
* Future




* CREATE
* INSERT
* UPDATE
* DELETE

* SELECT

(In general programming CRUD (Create,Read,Update,Delete))

# SQL Lite - Embedded SQL Database
###  https://www.sqlite.org/index.html
![SQL](https://www.sqlite.org/images/sqlite370_banner.gif)

* simple single user DB adhering to SQL standard
* up to 140TB in a single file

## SQLite vs other SQL databases (MySQL, Postegres, Oracle, Microsoft SQL server)
https://www.sqlite.org/whentouse.html

* https://www.mysql.com/ (Bought by Oracle, but still open source)
* MySQL fork: https://mariadb.org/
* https://www.postgresql.org/
* https://www.microsoft.com/en-us/sql-server

## Resources / books / tutorials / online viewers

### Books


 * https://www.amazon.com/Learning-SQL-Master-Fundamentals/dp/0596520832
 * https://www.amazon.com/Head-First-SQL-Brain-Learners/dp/0596526849
 * For manga lovers: https://www.amazon.com/Manga-Guide-Databases-Mana-Takahashi/dp/1593271905
 
Online
 * https://sqliteonline.com/

 * https://www.python-course.eu/sql_python.php      
 * [SchemaCrawler](https://www.schemacrawler.com/diagramming.html) - for crawling db schemas lot of yak shaving involved
 * https://www.danielms.site/blog/sqlite3-primer/
 * https://sqlzoo.net/ - great practice tool

In [ ]:
# Many ways to use SQL (command line, varius GUIs) ORM
# we'll use it embedded in Python

In [1]:
import sqlite3

In [2]:
db = sqlite3.connect('./db/chinook.db')

In [3]:
# prepare a cursor object using cursor() method
cursor = db.cursor()

In [28]:
# sample database Chinook
# fork at https://github.com/ValRCS/chinook-database
# good tutorials at http://www.sqlitetutorial.net/sqlite-python/

![CHINOOK STRUCTURE](sqlite-sample-database-color.jpg)

In [ ]:
# Open question on easiest way to visualize table schema: https://stackoverflow.com/questions/433071/good-tool-to-visualise-database-schema

## Chinook sample database tables
### There are 11 tables in the chinook sample database

*  employees table stores employees data such as employee id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
*  customers table stores customers data.
*  invoices & invoice_items tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
*  artists table stores artists data. It is a simple table that contains only artist id and name.
*  albums table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
*  media_types table stores media types such as MPEG audio file, ACC audio file, etc.
*  genres table stores music types such as rock, jazz, metal, etc.
*  tracks table store the data of songs. Each track belongs to one album.
*  playlists & playlist_track tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.

### SELECT DISTINCT column_list

### FROM table_list

###   JOIN table ON join_condition
  
### WHERE row_filter

### ORDER BY column

### LIMIT count OFFSET offset

### GROUP BY column

### HAVING group_filter

* Use ORDER BY clause to sort the result set
* Use DISTINCT clause to query unique rows in a table
* Use WHERE clause to filter rows in the result set
* Use LIMIT OFFSET clauses to constrain the number of rows returned
* Use INNER JOIN or LEFT JOIN to query data from multiple tables using join.
* Use GROUP BY to get the group rows into groups and apply aggregate function for each group.
* Use HAVING clause to filter groups

In [4]:
cursor.execute("SELECT * FROM artists");
results = cursor.fetchall()
type(results)

list

In [5]:
len(results)

275

In [6]:
results[:5]

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains')]

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [14]:
cursor.execute("""SELECT * FROM tracks
    WHERE milliseconds > 60*10*1000
    ORDER BY milliseconds DESC
    LIMIT 10;
""")
res = cursor.fetchall()
len(res)

10

In [3]:
import pandas as pd

In [4]:
df = pd.read_sql_query("SELECT * from invoices", db)
df.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [26]:
len(df)

412

In [6]:
composerdf = pd.read_sql_query("""SELECT composer, 
count(composer), 
ROUND(AVG(milliseconds)/1000,2) as sec,
MIN(milliseconds)/1000 shortie,
MAX(milliseconds)/1000 longestsong,
ROUND(SUM(unitprice),2) total
FROM tracks 
GROUP BY composer
--ORDER BY seconds DESC;
ORDER BY count(composer) DESC;""",db)
composerdf.head()

,Composer,count(composer),sec,shortie,longestsong,total
0,Steve Harris,80,340.21,105,678,79.20
1,U2,44,256.18,153,351,43.56
2,Jagger/Richards,35,247.63,49,479,34.65
3,Billy Corgan,31,255.24,161,473,30.69
4,Kurt Cobain,26,193.48,52,301,25.74


In [7]:
tracks = pd.read_sql_query("SELECT * FROM tracks", db)
tracks.head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [9]:
composergroup = tracks.groupby('Composer')
composergroup.head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
6,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
7,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
8,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
14,15,Go Down,4,1,1,AC/DC,331180,10847611,0.99


In [12]:
composergroup['Milliseconds']

In [27]:
tracks.groupby('Composer').count().sort_values(by="TrackId",ascending=False).head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes,UnitPrice
Composer,,,,,,,,
Steve Harris,80,80,80,80,80,80,80,80
U2,44,44,44,44,44,44,44,44
Jagger/Richards,35,35,35,35,35,35,35,35
Billy Corgan,31,31,31,31,31,31,31,31
Kurt Cobain,26,26,26,26,26,26,26,26


In [29]:
tracks.groupby('Composer').mean().sort_values(by="Milliseconds",ascending=False).head()

,TrackId,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes,UnitPrice
Composer,,,,,,,
Blackmore/Gillan/Glover/Lord/Paice,620.0,50.0,1.0,1.0,1196094.0,39267613.0,0.99
Jimmy Page/Led Zeppelin,1581.0,127.0,1.0,1.0,1116734.0,36052247.0,0.99
Gillan/Glover/Lord/Nix - Blackmore/Paice,621.0,50.0,1.0,1.0,913658.0,29846063.0,0.99
John Bonham/John Paul Jones/Robert Plant/Willie Dixon,1670.0,138.0,1.0,1.0,863895.0,28191437.0,0.99
Blackmore/Coverdale,622.0,50.0,1.0,1.0,854700.0,27775442.0,0.99


In [13]:
tracks.groupby('Composer')['Milliseconds'].count()

Composer
A. F. Iommi, W. Ward, T. Butler, J. Osbourne                     3
A. Jamal                                                         1
A.Bouchard/J.Bouchard/S.Pearlman                                 1
A.Isbell/A.Jones/O.Redding                                       1
AC/DC                                                            8
Aaron Copland                                                    1
Aaron Goldberg                                                   1
Ace Frehley                                                      2
Acyi Marques/Arlindo Bruz/Braço, Beto Sem/Zeca Pagodinho         1
Acyr Marques/Arlindo Cruz/Franco                                 1
Adalto Magalha/Lourenco                                          1
Adam Clayton, Bono, Larry Mullen & The Edge                     11
Adam Clayton, Bono, Larry Mullen, The Edge                      11
Addie William Jones/Nat Jones                                    1
Adrian Smith                                         

In [8]:
tracks.groupby('Composer').describe().T

Composer            A. F. Iommi, W. Ward, T. Butler, J. Osbourne    A. Jamal  \
TrackId      count                                  3.000000e+00        1.00   
             mean                                   2.108000e+03     1908.00   
             std                                    1.000000e+00         NaN   
             min                                    2.107000e+03     1908.00   
             25%                                    2.107500e+03     1908.00   
             50%                                    2.108000e+03     1908.00   
             75%                                    2.108500e+03     1908.00   
             max                                    2.109000e+03     1908.00   
AlbumId      count                                  3.000000e+00        1.00   
             mean                                   1.740000e+02      157.00   
             std                                    0.000000e+00         NaN   
             min                                    1.740000e+02      157.00   
             25%                                    1.740000e+02      157.00   
             50%                                    1.740000e+02      157.00   
             75%                                    1.740000e+02      157.00   
             max                                    1.740000e+02      157.00   
MediaTypeId  count                                  3.000000e+00        1.00   
             mean                                   1.000000e+00        1.00   
             std                                    0.000000e+00         NaN   
             min                                    1.000000e+00        1.00   
             25%                                    1.000000e+00        1.00   
             50%                                    1.000000e+00        1.00   
             75%                                    1.000000e+00        1.00   
             max                                    1.000000e+00        1.00   
GenreId      count                                  3.000000e+00        1.00   
             mean                                   3.000000e+00        2.00   
             std                                    0.000000e+00         NaN   
             min                                    3.000000e+00        2.00   
             25%                                    3.000000e+00        2.00   
             50%                                    3.000000e+00        2.00   
             75%                                    3.000000e+00        2.00   
             max                                    3.000000e+00        2.00   
Milliseconds count                                  3.000000e+00        1.00   
             mean                                   2.351797e+05   276871.00   
             std                                    1.055787e+05         NaN   
             min                                    1.721200e+05   276871.00   
             25%                                    1.742360e+05   276871.00   
             50%                                    1.763520e+05   276871.00   
             75%                                    2.667095e+05   276871.00   
             max                                    3.570670e+05   276871.00   
Bytes        count                                  3.000000e+00        1.00   
             mean                                   7.655451e+06  8980400.00   
             std                                    3.439761e+06         NaN   
             min                                    5.609799e+06  8980400.00   
             25%                                    5.669806e+06  8980400.00   
             50%                                    5.729813e+06  8980400.00   
             75%                                    8.678276e+06  8980400.00   
             max                                    1.162674e+07  8980400.00   
UnitPrice    count                                  3.000000e+00        1.00   


In [30]:
albums = pd.read_sql_query("""SELECT albums.title,composer, 
count(composer), 
ROUND(AVG(milliseconds)/1000,2) as sec,
MIN(milliseconds)/1000 shortie,
MAX(milliseconds)/1000 longestsong,
ROUND(SUM(milliseconds)/1000, 2) recordlen,
ROUND(SUM(unitprice),2) total
FROM tracks 
INNER JOIN albums
ON tracks.albumid = albums.albumid
GROUP BY albums.title
--ORDER BY seconds DESC;
--ORDER BY count(composer) DESC;
ORDER BY albums.title;""",db)
albums.head()

,Title,Composer,count(composer),sec,shortie,longestsong,recordlen,total
0,...And Justice For All,"James Hetfield, Lars Ulrich and Cliff Burton",9,436.55,313,588,3928.0,8.91
1,20th Century Masters - The Millennium Collecti...,None,0,287.37,205,390,3448.0,11.88
2,"A Copland Celebration, Vol. I",Aaron Copland,1,198.06,198,198,198.0,0.99
3,A Matter of Life and Death,None,0,432.29,258,564,4755.0,10.89
4,A Real Dead One,Steve Harris,12,298.93,228,471,3587.0,11.88


In [33]:
albums = pd.read_sql_query("""SELECT albums.title,artists.name,composer, 
count(composer), 
ROUND(AVG(milliseconds)/1000,2) as sec,
MIN(milliseconds)/1000 shortie,
MAX(milliseconds)/1000 longestsong,
ROUND(SUM(milliseconds)/1000, 2) recordlen,
ROUND(SUM(unitprice),2) total
FROM tracks 
INNER JOIN albums
ON tracks.albumid = albums.albumid
INNER JOIN artists
ON artists.artistid = albums.artistid
GROUP BY albums.title
--ORDER BY seconds DESC;
--ORDER BY count(composer) DESC;
ORDER BY albums.title;""",db)
albums.head()

,Title,Name,Composer,count(composer),sec,shortie,longestsong,recordlen,total
0,...And Justice For All,Metallica,"James Hetfield, Lars Ulrich and Cliff Burton",9,436.55,313,588,3928.0,8.91
1,20th Century Masters - The Millennium Collecti...,Scorpions,None,0,287.37,205,390,3448.0,11.88
2,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,Aaron Copland,1,198.06,198,198,198.0,0.99
3,A Matter of Life and Death,Iron Maiden,None,0,432.29,258,564,4755.0,10.89
4,A Real Dead One,Iron Maiden,Steve Harris,12,298.93,228,471,3587.0,11.88


In [17]:
tracks = pd.read_sql_query("""
SELECT name, composer, albums.title AS album, milliseconds/1000 as secs FROM tracks 
INNER JOIN albums
ON tracks.albumid = albums.albumid""",db)
tracks.head()

,Name,Composer,album,secs
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,343
1,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,205
2,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,233
3,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,210
4,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,203


In [20]:
songs = pd.read_sql_query("""
SELECT t.name, composer, a.title AS Album, art.name as Artist, milliseconds/1000 as secs FROM tracks AS t 
INNER JOIN albums AS a
ON t.albumid = a.albumid
-- DB comment we are joining 3 tables
INNER JOIN artists AS art
ON a.artistid = art.artistid""",db)
songs.head()

,Name,Composer,Album,Artist,secs
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,AC/DC,343
1,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,AC/DC,205
2,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,AC/DC,233
3,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,AC/DC,210
4,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,AC/DC,203


In [23]:
buyers = pd.read_sql_query("SELECT invoices.*, \
        c.FirstName, c.LastName FROM invoices \
        JOIN customers AS c \
        ON (invoices.CustomerID = c.CustomerID)\
        ORDER BY LastName", db)
buyers.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,FirstName,LastName
0,34,12,2009-05-23 00:00:00,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,0.99,Roberto,Almeida
1,155,12,2010-11-14 00:00:00,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,1.98,Roberto,Almeida
2,166,12,2010-12-25 00:00:00,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,13.86,Roberto,Almeida
3,221,12,2011-08-25 00:00:00,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,8.91,Roberto,Almeida
4,350,12,2013-03-31 00:00:00,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,1.98,Roberto,Almeida


In [26]:
buyers[buyers['CustomerId'] == 12].sum()

InvoiceId                                                         1694
CustomerId                                                          84
InvoiceDate          2009-05-23 00:00:002010-11-14 00:00:002010-12-...
BillingAddress       Praça Pio X, 119Praça Pio X, 119Praça Pio X, 1...
BillingCity          Rio de JaneiroRio de JaneiroRio de JaneiroRio ...
BillingState                                            RJRJRJRJRJRJRJ
BillingCountry              BrazilBrazilBrazilBrazilBrazilBrazilBrazil
BillingPostalCode    20040-02020040-02020040-02020040-02020040-0202...
Total                                                            37.62
FirstName            RobertoRobertoRobertoRobertoRobertoRobertoRoberto
LastName             AlmeidaAlmeidaAlmeidaAlmeidaAlmeidaAlmeidaAlmeida
dtype: object

In [24]:
buyers.groupby(['CustomerId']).sum()

,InvoiceId,Total
CustomerId,,
1,1582,39.62
2,1029,37.62
3,1715,39.62
4,1162,39.62
5,1435,40.62
6,1708,49.62
7,1568,42.62
8,1428,37.62
9,1288,37.62


In [27]:
songs = pd.read_sql_query("SELECT tracks.name, tracks.composer, tracks.unitPrice,\
        invoice_items.quantity, albums.title, \
        genres.name\
        FROM tracks \
        JOIN invoice_items \
        ON (tracks.trackid = invoice_items.trackid)\
        JOIN albums \
        ON (tracks.albumID = albums.albumID)\
        JOIN genres\
        ON (tracks.genreID = genres.genreID)", db)
songs.head(35)

,Name,Composer,UnitPrice,Quantity,Title,Name
0,Balls to the Wall,None,0.99,1,Balls to the Wall,Rock
1,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99,1,Restless and Wild,Rock
2,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",0.99,1,For Those About To Rock We Salute You,Rock
3,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",0.99,1,For Those About To Rock We Salute You,Rock
4,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",0.99,1,For Those About To Rock We Salute You,Rock
5,Breaking The Rules,"Angus Young, Malcolm Young, Brian Johnson",0.99,1,For Those About To Rock We Salute You,Rock
6,Dog Eat Dog,AC/DC,0.99,1,Let There Be Rock,Rock
7,Overdose,AC/DC,0.99,1,Let There Be Rock,Rock
8,Love In An Elevator,"Steven Tyler, Joe Perry",0.99,1,Big Ones,Rock
9,Janie's Got A Gun,"Steven Tyler, Tom Hamilton",0.99,1,Big Ones,Rock


In [32]:
songs['Quantity'].max()

1

In [27]:
len(buyers)

412

In [29]:
# we can do groupby on some column(s) then aggregrate some function(s) on columns that we want
df.groupby(['BillingCountry']).agg({'Total' : ['sum', 'count', 'min', 'max'], 'CustomerId' : 'count'})

Total                    CustomerId
                   sum count   min    max      count
BillingCountry                                      
Argentina        37.62     7  0.99  13.86          7
Australia        37.62     7  0.99  13.86          7
Austria          42.62     7  0.99  18.86          7
Belgium          37.62     7  0.99  13.86          7
Brazil          190.10    35  0.99  13.86         35
Canada          303.96    56  0.99  13.86         56
Chile            46.62     7  0.99  17.91          7
Czech Republic   90.24    14  0.99  25.86         14
Denmark          37.62     7  0.99  13.86          7
Finland          41.62     7  0.99  13.86          7
France          195.10    35  0.99  16.86         35
Germany         156.48    28  0.99  14.91         28
Hungary          45.62     7  0.99  21.86          7
India            75.26    13  1.98  13.86         13
Ireland          45.62     7  0.99  21.86          7
Italy            37.62     7  0.99  13.86          7
Netherlands      40.62     7  0.99  13.86          7
Norway           39.62     7  0.99  15.86          7
Poland           37.62     7  0.99  13.86          7
Portugal         77.24    14  0.99  13.86         14
Spain            37.62     7  0.99  13.86          7
Sweden           38.62     7  0.99  13.86          7
USA             523.06    91  0.99  23.86         91
United Kingdom  112.86    21  0.99  13.86         21

In [19]:
df.groupby(['BillingCountry']).sum()

,InvoiceId,CustomerId,Total
BillingCountry,,,
Argentina,1729,392,37.62
Australia,1043,385,37.62
Austria,1568,49,42.62
Belgium,1428,56,37.62
Brazil,7399,329,190.10
Canada,11963,1309,303.96
Chile,1176,399,46.62
Czech Republic,3143,77,90.24
Denmark,1288,63,37.62


In [30]:
cntry = pd.read_sql_query("SELECT BillingCountry, SUM(Total), AVG(Total), COUNT(Total), MAX(Total), MIN(Total), BillingCity, Total FROM invoices GROUP BY BillingCountry", db)
cntry

,BillingCountry,SUM(Total),AVG(Total),COUNT(Total),MAX(Total),MIN(Total),BillingCity,Total
0,Argentina,37.62,5.374286,7,13.86,0.99,Buenos Aires,0.99
1,Australia,37.62,5.374286,7,13.86,0.99,Sidney,0.99
2,Austria,42.62,6.088571,7,18.86,0.99,Vienne,0.99
3,Belgium,37.62,5.374286,7,13.86,0.99,Brussels,0.99
4,Brazil,190.10,5.431429,35,13.86,0.99,Rio de Janeiro,0.99
5,Canada,303.96,5.427857,56,13.86,0.99,Yellowknife,0.99
6,Chile,46.62,6.660000,7,17.91,0.99,Santiago,0.99
7,Czech Republic,90.24,6.445714,14,25.86,0.99,Prague,0.99
8,Denmark,37.62,5.374286,7,13.86,0.99,Copenhagen,0.99
9,Finland,41.62,5.945714,7,13.86,0.99,Helsinki,0.99


In [29]:
# most popular type of query!
table = pd.read_sql_query("SELECT * from albums", db)

In [20]:
table

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


In [21]:
tracks = pd.read_sql_query("""
    SELECT trackid,
        name,
         composer,
         unitprice
    FROM
     tracks;
    """, db)


In [22]:
tracks.head()

,TrackId,Name,Composer,UnitPrice
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",0.99
1,2,Balls to the Wall,None,0.99
2,3,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",0.99
3,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99
4,5,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,0.99


In [ ]:
# Exercise !
# Create full tables for playlists and customers

In [31]:
tracklen = pd.read_sql_query("""
    SELECT
     name,
     milliseconds, 
     albumid
    FROM
     tracks
    ORDER BY
     milliseconds DESC;
    """, db)
tracklen.head()

,Name,Milliseconds,AlbumId
0,Occupation / Precipice,5286953,227
1,Through a Looking Glass,5088838,229
2,"Greetings from Earth, Pt. 1",2960293,253
3,The Man With Nine Lives,2956998,253
4,"Battlestar Galactica, Pt. 2",2956081,253


In [34]:
alltracks = pd.read_sql_query("""
    SELECT
     trackid,
     name,
     title
    FROM
     tracks
    INNER JOIN albums ON albums.albumid = tracks.albumid;
    INNER JOIN albums ON albums.albumid = tracks.albumid;
    """, db)
alltracks.tail()


,TrackId,Name,Title
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music
3502,3503,Koyaanisqatsi,Koyaanisqatsi (Soundtrack from the Motion Pict...


In [36]:
tracksart = pd.read_sql_query("""
    SELECT
     trackid,
     tracks.name TRACK,
     Title,
     artists.name Artist,
     milliseconds/1000 AS Seconds
    FROM
     tracks
    INNER JOIN albums ON albums.albumid = tracks.albumid
    INNER JOIN artists ON albums.artistid = artists.artistid
    ORDER BY Seconds;
    """, db)
tracksart.tail()


,TrackId,TRACK,Title,Artist,Seconds
3498,3227,"Battlestar Galactica, Pt. 2","Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),2956
3499,3242,The Man With Nine Lives,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),2956
3500,3244,"Greetings from Earth, Pt. 1","Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),2960
3501,3224,Through a Looking Glass,"Lost, Season 3",Lost,5088
3502,2820,Occupation / Precipice,"Battlestar Galactica, Season 3",Battlestar Galactica,5286


In [37]:
tracksart.head()

,TrackId,TRACK,Title,Artist,Seconds
0,2461,É Uma Partida De Futebol,O Samba Poconé,Skank,1
1,168,Now Sports,Body Count,Body Count,4
2,170,A Statistic,Body Count,Body Count,6
3,178,Oprah,Body Count,Body Count,6
4,3304,Commercial 1,House of Pain,House Of Pain,7


In [39]:
# Exercise add Track length to our query